# Papermill Sampler Operation

__Description__: Tool to calculate the mean flow frequency curve, specify a range of annual exceedance probabilities with corresponding weights, and assign those weights to discharge events.

__Input__: [HEC-SSP](https://www.hec.usace.army.mil/software/hec-ssp/) .rpt files containing flow frequency data for a specific USGS Stream Gage calculated at a range of confidence limits and the following notebooks: `SSP_to_Mean_Curve`, `Stratified_Sampler`, and `Make_Production_Run_List`.

__Output__: Excel workbook with a sheet for the mean flow frequency curve table, the annual exceedance probability (AEP) vs weight table, and the discharge verses weight table.

---
## Load Libraries and Parameters:
### Libraries:

In [1]:
import sys
sys.path.append('../../core')
from meanffc import*

### Parameters:
#### Global:

In [2]:
## Filenames and paths:
gage_ID = '01134500'  # The USGS Station ID whose mean flow frequency curve will be calculated using this notebook.

root_dir = pl.Path(os.getcwd())       # The root directory/path
inputs_dir = root_dir/'Inputs'        # The directory/path where the SSP files can be found within the root directory
outputs_dir = root_dir/'Outputs'      # The directory/path to save the outputs
notebooks_dir = root_dir/'Notebooks'  # The directory/path to save the executed notebooks

## Options:
verbose = True      # Option for displaying print statements
round_decimals = 1  # Number of decimal places to include in the final mean curve table.

#### Notebook specific:

In [3]:
## SSP to Mean Curve:
version = '2_2'       # The version of HEC-SSP used to calculate the .rpt files. Either 2.1 or 2.2, specified as 2_1 or 2_2 here, respectively.
max_cl = 0.99       # Specify the upper confidence limit to restrict the confidence limit range, i.e. set the upper limit to 0.99 to restrict the range to 0.01 to 0.99.
adj_flows = 1.0       # [cfs]; amount to adjust the discharge for each annual exceedance probability if the discharge does not increase with decreasing probability.
Extrapolate = True    # True indicates that you want to extrapolate while False indicates that you want to only interpolate within the bounds of the minimum and maximum discharge for a specific CL. 
exclude_tails = True  # If True than the integration to calculate the mean curve only includes the area between the lower and upper confidence limits.

## Stratified Sampler:
min_val = 2     # The minimum recurrence interval
max_val = 3000  # The maximum recurrence interval
nbin = 100      # The number of recurrence intervals/annual exceedance probabilities 

#### Make directories if they do not exist:

In [4]:
dir_lst = [inputs_dir, outputs_dir, notebooks_dir]
make_directories(dir_lst, verbose)

C:\Users\sputnam\Desktop\MeanFFC\Inputs - already exists
C:\Users\sputnam\Desktop\MeanFFC\Outputs - created
C:\Users\sputnam\Desktop\MeanFFC\Notebooks - created


---
## Run Each Notebook:
### 1) SSP to Mean Curve:
#### Execute:

In [5]:
nb1_name = 'SSP_to_Mean_Curve'

nb1_path = pl.Path(notebooks_dir/'{0}_{1}.ipynb'.format(nb1_name, gage_ID))

nb1_parameters = {'gage_ID': gage_ID, 'inputs_dir': str(inputs_dir), 'outputs_dir': str(outputs_dir), 'version': version, 'max_cl': max_cl, 
                  'adj_flows': adj_flows, 'round_decimals': round_decimals, 'verbose': verbose, 'Extrapolate': Extrapolate, 'exclude_tails': exclude_tails}

pm.execute_notebook('{0}.ipynb'.format(nb1_name), str(nb1_path), nb1_parameters);

#### Display results:

In [6]:
nb1 = sb.read_notebook(str(nb1_path))

mean_curve = format_mean_curve(nb1.scraps['mean_curve'][1], verbose)

,Q_Mean_cfs,Q_Median_cfs
AEP,,
0.9,1431.9,1433.3
0.5,2095.3,2083.5


### 2) Stratified Sampler:
#### Execute:

In [7]:
nb2_name = 'Stratified_Sampler'

nb2_path = pl.Path(notebooks_dir/'{0}_{1}.ipynb'.format(nb2_name, gage_ID))

nb2_parameters = {'gage_ID': gage_ID, 'outputs_dir': str(outputs_dir), 'min_val': min_val, 'max_val': max_val, 'nbin': nbin, 'verbose': verbose}

pm.execute_notebook('{0}.ipynb'.format(nb2_name), str(nb2_path), nb2_parameters);

#### Display results:

In [8]:
nb2 = sb.read_notebook(str(nb2_path))

AEP_Weights = pd.DataFrame.from_dict(nb2.scraps['AEP_Weights'][1])

AEP_Weights.head(2)

,AEP,Weight
E0001,0.482369,0.035594
E0002,0.448352,0.032774


### 3) Make Production Run List: 
#### Execute:

In [9]:
nb3_name = 'Make_Production_Run_List'

nb3_path = pl.Path(notebooks_dir/'{0}_{1}.ipynb'.format(nb3_name, gage_ID))

nb3_parameters = {'gage_ID': gage_ID, 'outputs_dir': str(outputs_dir), 'round_decimals': round_decimals, 
                  'verbose': verbose, 'mean_curve': mean_curve.to_dict(), 'AEP_Weights': AEP_Weights.to_dict()}

pm.execute_notebook('{0}.ipynb'.format(nb3_name), str(nb3_path), nb3_parameters);

#### Display results:

In [10]:
nb3 = sb.read_notebook(str(nb3_path))

ProRunList = pd.DataFrame.from_dict(nb3.scraps['ProRunList'][1])

ProRunList.head(2)

,AEP,Q_Mean_cfs,Q_Median_cfs,RI,Weight
E0001,0.482369,2127.0,2115.0,2.073100,0.035594
E0002,0.448352,2189.6,2177.4,2.230391,0.032774


---
## Save the Results:

In [11]:
with pd.ExcelWriter(outputs_dir/'Sampler_Operation_{0}.xlsx'.format(gage_ID)) as writer:
    mean_curve.to_excel(writer, sheet_name='Mean Curve') 
    AEP_Weights.to_excel(writer, sheet_name='AEP Weights')
    ProRunList.to_excel(writer, sheet_name='Production Run List')
    
! jupyter nbconvert $nb1_path --log-level WARN
! jupyter nbconvert $nb2_path --log-level WARN
! jupyter nbconvert $nb3_path --log-level WARN

C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/scrapbook.scrap.json+json']) is not able to be represented.
  mimetypes=output.keys())
C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  mimetypes=output.keys())
C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/scrapbook.scrap.json+json']) is not able to be represented.
  mimetypes=output.keys())
C:\Users\sputnam\AppData\Local\Continuum\anaconda3\lib\site-packages\nbconvert\filters\datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['application/scrapbook.scrap.json+json']) is not able t

---
## End